In [1]:
import pandas as pd
import numpy as np
from haversine import haversine

In [2]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    
    data = {}
    depot_latitude = 40.748817
    depot_longitude = -73.985428
    customer_count = 10
    np.random.seed(seed=777)
    df = pd.DataFrame( {
                       "latitude":np.random.normal(depot_latitude, 0.007, customer_count),
                       "longitude":np.random.normal(depot_longitude, 0.007, customer_count)})
    distances = []
    locations = np.insert(df[['latitude','longitude']].values, 0, values=[depot_latitude,depot_longitude], axis=0)
    for i in locations:
        distances.append([haversine(i, j)*1000 for j in locations])
    data['locations'] = locations
    data['distance_matrix'] = distances
    np.random.seed(seed=777)
    demands = list(np.random.randint(4, 6, 10))
    demands.insert(0,0)
    data['demands'] = demands
    data['vehicle_capacities'] = [15, 15, 15, 15]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    res = {}
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        plan_seq = []
        xy_seq = []
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            plan_seq.append(node_index)
            xy_seq.append(list(data['locations'][node_index]))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_seq)
        res[vehicle_id] = xy_seq
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    rr = []
    for x in res:
        rr.append(res[x])
    return rr


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        res = print_solution(data, manager, routing, solution)
    return res,data

In [3]:
res,data = main()

Objective: 10321
[0, 10]
Route for vehicle 0:
 0 Load(0) ->  10 Load(4) ->  0 Load(4)
Distance of the route: 1590m
Load of the route: 4

[0, 6, 5, 7]
Route for vehicle 1:
 0 Load(0) ->  6 Load(5) ->  5 Load(10) ->  7 Load(15) ->  0 Load(15)
Distance of the route: 2127m
Load of the route: 15

[0, 1, 2, 9]
Route for vehicle 2:
 0 Load(0) ->  1 Load(5) ->  2 Load(10) ->  9 Load(15) ->  0 Load(15)
Distance of the route: 3719m
Load of the route: 15

[0, 3, 4, 8]
Route for vehicle 3:
 0 Load(0) ->  3 Load(5) ->  4 Load(9) ->  8 Load(14) ->  0 Load(14)
Distance of the route: 2885m
Load of the route: 14

Total distance of all routes: 10321m
Total load of all routes: 48


In [4]:
from ipyleaflet import Map, basemaps, basemap_to_tiles
from ipyleaflet import Marker

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center = (40.748817,-73.985428),
    zoom=13
    )
for lat,lon in data['locations']:
    marker = Marker(location=(lat, lon), draggable=False)
    m.add_layer(marker)
m
from ipyleaflet import Polyline
poly_line = Polyline(locations=res, color="dodgerblue" , fill=False, weight=2)

m.add_layer(poly_line)
m.layout.height="550px"

m

Map(center=[40.748817, -73.985428], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…